# 03 — Fine-tune sentiment model (weak supervision + bag regularizer)

We’ll fine-tune `distilbert-base-uncased` on your **English YouTube comments**, using:

* **Weak labels** from the video “vibe” (negative/positive), and
* A **bag-level KL term** that nudges the *average* comment sentiment per video toward the video-level target.

Outputs are saved to `models/sentiment_en/` for your Streamlit app.

---

## A) Setup & load splits

> Loads train/val/test from step **02** and shows class counts.

In [1]:
# Imports & paths
from pathlib import Path
import numpy as np
import pandas as pd
import torch, torch.nn as nn
import transformers, evaluate, sklearn

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

print("transformers:", transformers.__version__)
DATA_DIR = Path("../data")
OUT_DIR  = Path("../..") / "models" / "sentiment_en"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Load data
df_tr = pd.read_csv(DATA_DIR / "sentiment_train.csv")
df_va = pd.read_csv(DATA_DIR / "sentiment_val.csv")
df_te = pd.read_csv(DATA_DIR / "sentiment_test.csv")

for name, df in [("train", df_tr), ("val", df_va), ("test", df_te)]:
    print(name, df["sentiment"].value_counts(), "\n")

transformers: 4.55.2
train sentiment
negative    783
positive    679
Name: count, dtype: int64 

val sentiment
negative    168
positive    145
Name: count, dtype: int64 

test sentiment
negative    168
positive    146
Name: count, dtype: int64 



---

## B) Prep: labels, per-video IDs, weak targets

> Collapses 3-way targets (neg/neu/pos) into **binary** (neg/pos) proportions; assigns a stable `vid` per `video_id`.

In [2]:
# Label maps
label2id = {"negative":0, "positive":1}
id2label = {v:k for k,v in label2id.items()}

# Collapse targets to 2D (neg,pos)
def to_2d_target(row):
    tn = float(row.get("target_neg", 0.0))
    tp = float(row.get("target_pos", 0.0))
    s = tn + tp
    if s <= 0:
        return pd.Series([0.5, 0.5], index=["t_neg","t_pos"])
    return pd.Series([tn/s, tp/s], index=["t_neg","t_pos"])

# Prepare dataframes
def prep(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["labels"] = df["sentiment"].map(label2id).astype(int)
    # stable int IDs for videos
    vid_map = {v:i for i,v in enumerate(df["video_id"].astype(str).unique())}
    df["vid"] = df["video_id"].astype(str).map(vid_map).astype(int)
    # 2-d per-row target (weak supervision)
    df[["t_neg","t_pos"]] = df.apply(to_2d_target, axis=1)
    return df

df_tr = prep(df_tr)
df_va = prep(df_va)
df_te = prep(df_te)

df_tr.head(3)

,text,sentiment,video_id,target_neg,target_neu,target_pos,like_count,source_url,labels,vid,t_neg,t_pos
0,Boring this new snow white ruin Disney instead...,negative,iV46TJKL8cU,0.80,0.15,0.05,0,https://www.youtube.com/watch?v=iV46TJKL8cU,0,0,0.941176,0.058824
1,The September 11th attacks have been forgotten...,negative,iV46TJKL8cU,0.80,0.15,0.05,1,https://www.youtube.com/watch?v=iV46TJKL8cU,0,0,0.941176,0.058824
2,She is jealous i think... i wish i can watch i...,positive,n0OFH4xpPr4,0.05,0.15,0.80,0,https://www.youtube.com/watch?v=n0OFH4xpPr4,1,1,0.058824,0.941176


---

## C) Tokenize & build HF datasets

> Keeps custom fields (`vid`, `tgt2`) for the custom loss; we’ll tensorize them in a custom collator.

In [3]:
# Tokenizer
MODEL = "distilbert-base-uncased"
tok = AutoTokenizer.from_pretrained(MODEL)
max_len = 160

# Tokenize + carry extras
def tok_fn(batch):
    enc = tok(batch["text"], truncation=True, padding="max_length", max_length=max_len)
    enc["labels"] = batch["labels"]
    enc["vid"]    = batch["vid"]
    # ensure list-of-lists, not tuples → safer tensorization
    enc["tgt2"]   = [[float(a), float(b)] for a, b in zip(batch["t_neg"], batch["t_pos"])]
    return enc

# Build datasets
ds = DatasetDict({
    "train": Dataset.from_pandas(df_tr[["text","labels","vid","t_neg","t_pos"]]),
    "val":   Dataset.from_pandas(df_va[["text","labels","vid","t_neg","t_pos"]]),
    "test":  Dataset.from_pandas(df_te[["text","labels","vid","t_neg","t_pos"]]),
}).map(tok_fn, batched=True, remove_columns=["text","t_neg","t_pos"])

# Leave as python lists; our collator will create tensors (and preserve extras)
for split in ds:
    ds[split].set_format(type=None)

len(ds["train"]), len(ds["val"]), len(ds["test"])

Map:   0%|          | 0/1462 [00:00<?, ? examples/s]

Map:   0%|          | 0/313 [00:00<?, ? examples/s]

Map:   0%|          | 0/314 [00:00<?, ? examples/s]

(1462, 313, 314)

---

## D) Model, class weights, and custom Trainer (bag-level KL)

> The loss = **CE on weak labels** + **β · KL(avg_pred_per_video || target_per_video)** within each batch.

In [4]:
# Class weights for noisy labels
y = df_tr["labels"].values
classes = np.array([0,1])
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y)
class_weights = torch.tensor(cw, dtype=torch.float)

# Base model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL, num_labels=2, id2label=id2label, label2id=label2id
)

# Custom Trainer with bag-level KL
class BagKLBinaryTrainer(Trainer):
    def __init__(self, *args, beta=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        self.beta = beta
        self.ce = nn.CrossEntropyLoss(weight=class_weights)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # custom fields (preserved via our collator)
        labels = inputs.pop("labels")
        vids   = inputs.pop("vid")
        tgt2   = inputs.pop("tgt2")  # [B,2] float tensor

        outputs = model(**inputs)
        logits  = outputs.logits
        probs   = torch.softmax(logits, dim=-1)

        # (1) CE on weak labels
        loss_ce = self.ce(logits, labels)

        # (2) Bag-level KL per video in-batch
        kl = nn.KLDivLoss(reduction="batchmean")
        loss_agg = logits.new_tensor(0.0)
        uniq = torch.unique(vids)
        for v in uniq:
            m = (vids == v)
            if m.sum() == 0:
                continue
            avg_pred = probs[m].mean(dim=0).clamp(1e-6, 1.0)
            avg_tgt  = tgt2[m].mean(dim=0).clamp(1e-6, 1.0)
            loss_agg = loss_agg + kl(torch.log(avg_pred), avg_tgt)

        loss = loss_ce + self.beta * loss_agg
        return (loss, outputs) if return_outputs else loss

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---

## E) Metrics, collator, training args, trainer

> The **custom collator** pads token fields and re-attaches `vid`/`tgt2`.
> `remove_unused_columns=False` is crucial so Trainer won’t drop our extras.



In [5]:
# Metrics
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")
metric_pr  = evaluate.load("precision")
metric_rc  = evaluate.load("recall")

def compute_metrics(p):
    logits = p.predictions[0] if isinstance(p.predictions, (tuple, list)) else p.predictions
    y_pred = logits.argmax(-1)
    y_true = p.label_ids
    return {
        "acc":  metric_acc.compute(predictions=y_pred, references=y_true)["accuracy"],
        "f1":   metric_f1.compute(predictions=y_pred, references=y_true, average="binary")["f1"],
        "prec": metric_pr.compute(predictions=y_pred, references=y_true, average="binary")["precision"],
        "rec":  metric_rc.compute(predictions=y_pred, references=y_true, average="binary")["recall"],
    }

# Custom collator (preserves extras)
from transformers import DataCollatorWithPadding
_pad = DataCollatorWithPadding(tokenizer=tok)

def collate_with_extras(features):
    # token fields to pad
    token_keys = ["input_ids", "attention_mask"]
    if "token_type_ids" in features[0]:
        token_keys.append("token_type_ids")

    # split tokenizable vs. extras
    tok_feats = [{k: f[k] for k in token_keys if k in f} | {"labels": f["labels"]} for f in features]
    batch = _pad(tok_feats)

    # add our extras as tensors
    if "vid" in features[0]:
        batch["vid"]  = torch.tensor([f["vid"] for f in features], dtype=torch.long)
    if "tgt2" in features[0]:
        batch["tgt2"] = torch.tensor([f["tgt2"] for f in features], dtype=torch.float)
    return batch

# TrainingArguments & Trainer
args = TrainingArguments(
    output_dir=str(OUT_DIR / "ckpt"),
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=False,
    report_to="none",
    remove_unused_columns=False,   # 🔑 keep 'vid' and 'tgt2'
    seed=42,
)

trainer = BagKLBinaryTrainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["val"],
    data_collator=collate_with_extras,  # 🔑 preserves vid/tgt2
    tokenizer=tok,                      # ok to include; collator handles extras
    compute_metrics=compute_metrics,
    beta=0.5,
)

# Optional quick sanity check:
b = next(iter(trainer.get_train_dataloader()))
print("Batch keys:", list(b.keys()))  # expect: input_ids, attention_mask, labels, vid, tgt2

Batch keys: ['input_ids', 'attention_mask', 'labels', 'vid', 'tgt2']


C:\Users\kongj\AppData\Local\Temp\ipykernel_21104\386921117.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `BagKLBinaryTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


---

## F) Train → Eval → Save

> Saves model + tokenizer into `models/sentiment_en/` so your Streamlit app can load it directly.



In [6]:
# Train & evaluate
trainer.train()

print("VAL:", trainer.evaluate(ds["val"]))
print("TEST:", trainer.evaluate(ds["test"]))

# Save model & tokenizer
trainer.save_model(str(OUT_DIR))
tok.save_pretrained(str(OUT_DIR))
print("Saved to:", OUT_DIR)

Step,Training Loss
50,0.757200
100,0.428000
150,0.283800
200,0.206300
250,0.146800


VAL: {'eval_loss': 0.3150944709777832, 'eval_acc': 0.8785942492012779, 'eval_f1': 0.8689655172413793, 'eval_prec': 0.8689655172413793, 'eval_rec': 0.8689655172413793, 'eval_runtime': 12.7348, 'eval_samples_per_second': 24.578, 'eval_steps_per_second': 0.393, 'epoch': 3.0}
TEST: {'eval_loss': 0.3868599832057953, 'eval_acc': 0.8598726114649682, 'eval_f1': 0.8523489932885906, 'eval_prec': 0.8355263157894737, 'eval_rec': 0.8698630136986302, 'eval_runtime': 12.5844, 'eval_samples_per_second': 24.951, 'eval_steps_per_second': 0.397, 'epoch': 3.0}
Saved to: ..\..\models\sentiment_en


---

## G) Quick inference helper (local sanity test)

> Handy for spot-checking outputs before wiring into the app.

In [7]:
class SentimentEN:
    def __init__(self, path="../../models/sentiment_en"):
        self.tok = AutoTokenizer.from_pretrained(path)
        self.model = AutoModelForSequenceClassification.from_pretrained(path).eval()
        self.id2label = self.model.config.id2label

    def __call__(self, texts):
        enc = self.tok(texts, return_tensors="pt", truncation=True, padding=True, max_length=160)
        with torch.no_grad():
            logits = self.model(**enc).logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()
        labels = [self.id2label[int(i)] for i in probs.argmax(-1)]
        return labels, probs

---

## ✅ Wrap-up: What we just did (03_sentiment_finetune)

**TL;DR:** We fine-tuned DistilBERT on your YouTube comments with weak labels, plus a **bag-level KL** that aligns the average comment sentiment per video with the video’s overall vibe. Then we saved the model to plug straight into your Streamlit app.

**Highlights**

* Weak labels (neg/pos) from video-level tags
* **Class balancing** + **bag-level regularization (β=0.5)**
* Reproducible splits & seed
* Saved to: `models/sentiment_en/`